# Q4_Answer

运行 TwoDimDiffusion.zip 中的程序，体会、理解和掌握有限差分和有限体积
法的求解 2D 热传导方程的基本步骤。掌握输出给定点温度值的方法

## 第一步：初始化相关参数

In [14]:
# import numpy as np
col_len = 30  # 列数
row_len = 50  # 行数
node_num = col_len * row_len  # 网格数量
tlist_new =  [[20] * col_len for row in range(row_len)]  # 记录新温度
tlist_old =  [[20] * col_len for row in range(row_len)]  # 记录旧温度
# tlist_new =  np.array([[20] * col_len for row in range(row_len)])  # 记录新温度
# tlist_old =  np.array([[20] * col_len for row in range(row_len)])  # 记录旧温度
t_left = 100  # 左边界侧温度

max_loops = 10000  # 迭代次数
step_before_1w = 1000  # 迭代次数为 1w 前（含）步长
step_after_1w = 10000  # 迭代次数为 1w 后步长
loop_count = 0  # 迭代保存计数器

In [15]:
delta_x = 0.5
delta_t = 1
alpha = 0.005
alpha_dimless = alpha / (delta_x * delta_x / delta_t)

## 第二步：定义有限差分算法

In [16]:
# from numba import jit

# @jit(nopython=True)
def thermal_conductivity(tlist_old):
    # 去 左 右 下 边界区域
    for row in range(row_len-1):  # 去下边界
        for col in range(1, col_len-1):  # 去 右 左 边界
            tlist_new[row][col] = tlist_old[row][col] + alpha_dimless * \
                (tlist_old[row][col+1]+tlist_old[row][col-1] + tlist_old[row+1]
                 [col]+tlist_old[row-1][col] - 4 * tlist_old[row][col])
                 
    # 左边界
    for row in range(row_len):
        tlist_new[row][0] = t_left

    # 右边界: partial T / partial x = 0
    for row in range(row_len):
        tlist_new[row][col_len-1] = tlist_new[row][col_len-2]

    # 下边界
    for col in range(1, col_len-1):
        tlist_new[row][col] = tlist_old[row][col] + alpha_dimless * \
            (tlist_old[row][col+1]+tlist_old[row][col-1] + tlist_old[-1]
             [col]+tlist_old[row-1][col] - 4 * tlist_old[row][col])

    return tlist_new

## 第三步：启动循环

In [17]:
import copy
import time
print(time.time())
thermal_data = []
append = thermal_data.append
for loops in range(max_loops+1):
    tlist_old = thermal_conductivity(tlist_old)
    if loops % step_before_1w == 0:
        if loops <= 10000 or loops % step_after_1w == 0:
            append(copy.deepcopy(tlist_old))
            loop_count += 1
print(time.time())

1556293389.0557647
1556293397.3267856


## 第五步：绘图

In [18]:
# 导入绘图库
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

In [19]:
data = [go.Heatmap(visible = False,zmin =20, zmax = 100,z = t,hovertemplate="<extra>x: %{x}<Br>y: %{y}<Br>temp: %{z}</extra> ") for t in thermal_data]
data[0]['visible'] = True

In [20]:
steps = []
data_len = len(data)
for i in range(data_len):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * data_len],
        label = i
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active = 1,
    currentvalue = {"prefix": "loop: "},
    steps = steps
)]

layout = dict(sliders=sliders)

fig = dict(data=data, layout=layout)

plotly.offline.iplot(fig)

In [ ]:
data = [
    go.Surface(z=tlist_old,hovertemplate="<extra>x: %{x}<Br>y: %{y}<Br>temp: %{z}</extra> ")
]
data1 = [go.Heatmap(z=tlist_old,hovertemplate="<extra>x: %{x}<Br>y: %{y}<Br>temp: %{z}</extra> ")]

In [ ]:
plotly.offline.iplot(data)
plotly.offline.iplot(data1)